## Line Following with Obstacle Avoidance

The final task of this week is a combination of the line following, obstacle detection, and wall following behaviors.

The robot should follow the line until an obstacle is detected in its path. Then, the robot will turn right and follow the wall at its right until the line is detected again, and it will resume the line following behavior.

Please watch the following demo video:

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('Jd1jpt3pgc8')

This is the most complex task that we have programmed so far, thus it is a nice candidate for developing with the so-called [**top-down** approach](https://en.wikipedia.org/wiki/Top-down_and_bottom-up_design). With this methodology, we start with a high-level algorithm, and break it down into its components:

    repeat forever
        follow line until an obstacle is detected
        get close to the wall
        follow wall until a line is detected
        get close to the line

### Initialization
First, we need to import all the required modules.

In [1]:
import packages.initialization
import pioneer3dx as p3dx
p3dx.init()
import cv2
import numpy

In [2]:
def getSonars():
    rightSide = p3dx.distance[7]
    frontSide = min(p3dx.distance[3:5])
    return rightSide, frontSide

In [3]:
def line_centroid(image):
            hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
            lower_cyan = numpy.array([80, 100, 100])
            upper_cyan = numpy.array([100, 255, 255])
            mask = cv2.inRange(hsv, lower_cyan, upper_cyan)
            mask[0:70, 0:150] = 0
            M = cv2.moments(mask)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            return cx, cy

In [4]:
MIN_WALL_THRESHOLD = 0.35
MAX_WALL_THRESHOLD = 0.4

In [5]:
DEF_X_SPEED = 0.32       # default forward velocity
DEF_YAW_SPEED = 0.25    # default turning velocity

### Component functions
The first function must return `True` if an obstacle is detected in front of the robot, or `False` otherwise.

In [6]:
def is_obstacle_detected():
    threshold = 0.7 # not too far
    # True if front distance < threshold
    return (min(p3dx.distance[3:5]) < threshold)

The second function is the line following behavior as seen in previous notebooks during this week.

In [7]:
def follow_line():
    print('Following the line')
    while not is_obstacle_detected():
        try:
            width = p3dx.image.shape[1]
            cx, cy = line_centroid(p3dx.image)
            err = cx- (width/2)
            linear = 2
            angular = -(0.1*err)
            move(linear, angular)
        except is_obstacle_detected:
            move (0,0)
            print ('obstacle detected')

The next function was developed in the previous week: the robot turns until it is approximately parallel to the wall.

In [8]:
def getWall():
    rightSide, frontSide = getSonars()
    # move forward until an obstacle is detected at the front
    while (rightSide > MAX_WALL_THRESHOLD) and (frontSide > MAX_WALL_THRESHOLD):       
        move(DEF_X_SPEED,0)
        rightSide, frontSide = getSonars()
        
    # turn left until the wall is detected by right sonar 7
    while p3dx.distance[7] > MAX_WALL_THRESHOLD:
        # rotate more if we're almost bumping in front
        if min(rightSide,frontSide) == frontSide:
            yawSpeed = DEF_YAW_SPEED * 3
        else:
            yawSpeed = DEF_YAW_SPEED
        move(0,yawSpeed)
        rightSide, frontSide = getSonars()
    move(0,0)
    print "Wall found"

The next function is checked during the wall following behavior: it must return `True` when the line is again detected, or `False` otherwise.

In [9]:
def is_line_detected():
            width = p3dx.image.shape[1]
            hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
            lower_cyan = numpy.array([80, 100, 100])
            upper_cyan = numpy.array([100, 255, 255])
            mask = cv2.inRange(hsv, lower_cyan, upper_cyan)
            mask[0:80, 0:150] = 0
            M = cv2.moments(mask)
            if M['m00'] == 0:
                return False # no line
            else:
                return True  # line
            print ('line detected')

Next, we reuse the wall following behavior from previous week.

In [10]:
def follow_wall():
    print('Following the wall')
    while not is_line_detected():
        # by default, just move forward
        xSpeed = DEF_X_SPEED
        yawSpeed = 0
        rightSide, frontSide = getSonars()
        # if we're getting too close to the wall with the front side...
        if frontSide < MIN_WALL_THRESHOLD:
            # go backward and turn left quickly (x4)
            xSpeed = -0.1
            yawSpeed = DEF_YAW_SPEED * 4
        else:
            # if we're getting too close to the wall with the right side...
            if rightSide < MIN_WALL_THRESHOLD:
                # move slowly forward (x0.5) and turn left
                xSpeed  = DEF_X_SPEED * 0.5
                yawSpeed = DEF_YAW_SPEED
            else:
                # if we're getting too far away from the wall with the right side...
                if rightSide > MAX_WALL_THRESHOLD:
                    # move slowly forward (x0.5) and turn right
                    xSpeed  = DEF_X_SPEED * 0.5
                    yawSpeed = -DEF_YAW_SPEED
        # Move the robot
        move(xSpeed,yawSpeed)

Finally, a function is needed for turning the robot slightly until it is approximately parallel to the line again.

In [11]:
def getLine():
    print('Get the line ------')
    #get the coordinates of the centroid
    cx, cy = line_centroid(p3dx.image)
    width = p3dx.image.shape[1]
    # center the robot towards the centroid
    print('   Center the robot towards the centroid')
    while not (65 <= cx <= 85):
        #print ("cx %d, cy %d" % (cx, cy))
        cx, cy = line_centroid(p3dx.image)
        err = cx - (width / 2 )
        angular = - (0.05 / 4) * err
        move(0, angular)
    # if the aligniament is correct
    print('Move slowly ahead')  
    move(0.2, 0)
    p3dx.sleep(2)
    print('   Turn slowly to the left')
    move(0, 0.2)
    p3dx.sleep(5)
    move(0,0)
    print('   Robot on the line ---<8O<(')
#end of getLine()

Some additional lower-level functions are required (guess which ones?).

You can define them in the next empty cell.

In [12]:
# Lower-level functions
def move(V_robot,w_robot):
    r = 0.1953 / 2
    L = 0.33
    w_r = (2 * V_robot + L * w_robot) / (2*r)
    w_l = (2 * V_robot - L * w_robot) / (2*r)
    p3dx.move(w_l, w_r)

### Main loop
The main loop looks very similar to the proposed algorithm:

In [13]:
p3dx.tilt(-0.47)
try:
    while True:
        follow_line()
        getWall()
        follow_wall()
        getLine()
except KeyboardInterrupt:
    move(0,0)

Following the line
Wall found
Following the wall
Get the line ------
   Center the robot towards the centroid
Move slowly ahead
   Turn slowly to the left
   Robot on the line ---<8O<(
Following the line


Did it work? Congratulations, you have completed the task of this week!

---
#### Try-a-Bot: an open source guide for robot programming
Developed by:
[![Robotic Intelligence Lab @ UJI](img/logo/robinlab.png "Robotic Intelligence Lab @ UJI")](http://robinlab.uji.es)

Sponsored by:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="http://www.ieee-ras.org"><img src="img/logo/ras.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.cyberbotics.com"><img src="img/logo/cyberbotics.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.theconstructsim.com"><img src="img/logo/theconstruct.png"></a>
</td>
</tr>
</table>

Follow us:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="https://www.facebook.com/RobotProgrammingNetwork"><img src="img/logo/facebook.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="https://www.youtube.com/user/robotprogrammingnet"><img src="img/logo/youtube.png"></a>
</td>
</tr>
</table>